In [1]:
import os
import numpy as np
import pandas as pd
import warnings

# This is not recommended but I am doing this to suppress warnings from SARIMAX
warnings.simplefilter('ignore')

countryName = 'Germany'
nFeatures = 1

nDaysMin = 3

nValid = 10
nTest = 10

dataDir = os.path.join('data', 'JHU', 'upto05252020_forPublication')


confirmedFilename = 'time_series_covid19_confirmed_global.csv'
deathsFilename = 'time_series_covid19_deaths_global.csv'
recoveredFilename = 'time_series_covid19_recovered_global.csv'

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
def meanAbsolutePercentageError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    absPcErrorList = [absError/yTrue for absError, yTrue in zip(absErrorList, yTrueList)]
    MAPE = 100*np.mean(absPcErrorList)
    return MAPE

def meanForecastError(yTrueList, yPredList):
    forecastErrors = [yTrue - yPred for yTrue, yPred in zip(yTrueList, yPredList)]
    MFE = np.mean(forecastErrors)
    return MFE

def meanAbsoluteError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(absErrorList)

def meanSquaredError(yTrueList, yPredList):
    sqErrorList = [np.square(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(sqErrorList)

def rootMeanSquaredError(yTrueList, yPredList):
    return np.sqrt(meanSquaredError(yTrueList, yPredList))

def medianSymmetricAccuracy(yTrueList, yPredList):
    '''https://helda.helsinki.fi//bitstream/handle/10138/312261/2017SW001669.pdf?sequence=1'''
    logAccRatioList = [np.abs(np.log(yPred/yTrue)) for yTrue, yPred in zip(yTrueList, yPredList)]
    MdSA = 100*(np.exp(np.median(logAccRatioList))-1)
    return MdSA

In [4]:
# Function to get all three frames for a given country
def getCountryCovidFrDict(countryName):
    countryCovidFrDict = {}
    for key in covidFrDict.keys():
        dataFr = covidFrDict[key]
        countryCovidFrDict[key] = dataFr[dataFr['Country/Region'] == countryName]
    return countryCovidFrDict

In [5]:
# Load all 3 csv files
covidFrDict = {}
covidFrDict['confirmed'] = pd.read_csv(os.path.join(dataDir, confirmedFilename))
covidFrDict['deaths'] = pd.read_csv(os.path.join(dataDir, deathsFilename))

# Recovered is back again!
covidFrDict['recovered'] = pd.read_csv(os.path.join(dataDir, recoveredFilename))

countryCovidFrDict = getCountryCovidFrDict(countryName)

# Get list of dates
colNamesList = list(countryCovidFrDict['confirmed'])
dateList = [colName for colName in colNamesList if '/20' in colName]
dataList = [countryCovidFrDict['confirmed'][date].iloc[0] for date in dateList]
dataDict = dict(zip(dateList, dataList))

# Get time series for cases > 100 only
daysSince = 100
nCasesGreaterDaysSinceList = []
datesGreaterDaysSinceList = []

for key in dataDict.keys():
    if dataDict[key] > daysSince:
        datesGreaterDaysSinceList.append(key)
        nCasesGreaterDaysSinceList.append(dataDict[key])
        
XList, yList = split_sequence(nCasesGreaterDaysSinceList, nDaysMin)

XTrainList = XList[0:len(XList)-(nValid + nTest)]
XValidList = XList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
XTestList = XList[-nTest:]

yTrain = yList[0:len(XList)-(nValid + nTest)]
yValid = yList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
yTest = yList[-nTest:]

print('Total size of data points for LSTM:', len(yList))
print('Size of training set:', len(yTrain))
print('Size of validation set:', len(yValid))
print('Size of test set:', len(yTest))

# Convert from list to matrix
XTrain = XTrainList.reshape((XTrainList.shape[0], XTrainList.shape[1], nFeatures))
XValid = XValidList.reshape((XValidList.shape[0], XValidList.shape[1], nFeatures))
XTest = XTestList.reshape((XTestList.shape[0], XTestList.shape[1], nFeatures))

print(XTrain.shape)

Total size of data points for LSTM: 83
Size of training set: 63
Size of validation set: 10
Size of test set: 10
(63, 3, 1)


# Vanilla LSTM

In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

nNeurons = 100
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    
    # define model
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 3.582622974481685
Updating best MAPE to 3.582622974481685...
Updating best seed to 0...
1 4.108660619769815
2 5.355526328895277
3 3.456197576629796
Updating best MAPE to 3.456197576629796...
Updating best seed to 3...
4 2.222136839875857
Updating best MAPE to 2.222136839875857...
Updating best seed to 4...
5 99.88626912117176
6 0.8396448825954389
Updating best MAPE to 0.8396448825954389...
Updating best seed to 6...
7 0.3231927460285669
Updating best MAPE to 0.3231927460285669...
Updating best seed to 7...
8 0.846547159329296
9 1.3639559687309524
10 99.8871216852602
11 3.515872498792578
12 0.4214865744488002
13 3.293389350446922
14 5.67372947635369
15 0.42458661988563845
16 0.18107496412498136
Updating best MAPE to 0.18107496412498136...
Updating best seed to 16...
17 1.422320660356799
18 1.7459630969635134
19 1.3875471971893214
20 1.6917072246391096
21 1.87816837019033
22 0.4356156692517582
23 1.2949281156468109
24 2.01903642357567
25 99.88520731795639
26 3.0495147559661806
27 1.473

# Stacked LSTM

In [7]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 0.3983086111382627
Updating best MAPE to 0.3983086111382627...
Updating best seed to 0...
1 93.7547028929939
2 1.0061635701645562
3 0.8483920759226758
4 5.458054807832709
5 50.304759717058246
6 4.291231922197032
7 3.712840027208976
8 0.8491395282762477
9 3.2579084917957566
10 50.54698936744646
11 5.539934773797022
12 2.2341833636280026
13 1.6116159230994405
14 2.771097403826137
15 0.7683253910118575
16 1.6615396913555858
17 3.4125615652035965
18 4.905189352781322
19 3.08812148231429
20 0.29640426813713194
Updating best MAPE to 0.29640426813713194...
Updating best seed to 20...
21 1.3873419926654478
22 0.7192953004585405
23 50.2391965277123
24 2.9825139401702083
25 49.15603724699505
26 50.20245571551095
27 0.13180569452789018
Updating best MAPE to 0.13180569452789018...
Updating best seed to 27...
28 0.812890158686184
29 1.2662518012392159
30 52.170059696272716
31 0.4033871542231618
32 2.3507467887207265
33 50.1354032792048
34 1.541506014411862
35 4.25096489055263
36 4.96079561553683


# Bidirectional LSTM

In [8]:
from tensorflow.keras.layers import Bidirectional

# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 0.14513092886203569
Updating best MAPE to 0.14513092886203569...
Updating best seed to 0...
1 1.119639430548049
2 0.5591927298503951
3 0.14371703044809772
Updating best MAPE to 0.14371703044809772...
Updating best seed to 3...
4 0.15017816752221103
5 1.7616716002553152
6 1.2821783437315575
7 0.3055601467375408
8 2.926030813180094
9 5.052663166957912
10 4.660243216933865
11 0.9878777120209188
12 2.8733059332566526
13 0.22232265395270454
14 1.853819432721119
15 1.279526747971498
16 1.7742459182371322
17 0.1738283046118898
18 0.33362101987632076
19 0.16707874995376878
20 0.44421896935389027
21 99.94454263482527
22 0.28464687811639827
23 99.90347109590437
24 0.15525467914313107
25 1.477238712603165
26 1.8520136852919498
27 0.14268910311864125
Updating best MAPE to 0.14268910311864125...
Updating best seed to 27...
28 12.387673263127523
29 1.450506316053386
30 0.8975865542951869
31 0.8516085690322516
32 0.6342003456085155
33 1.6712338105228062
34 0.5043516111922708
35 6.092824352906342
36

# CNN LSTM

In [9]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.compat.v1.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

# Number of subsequences to break X into (we do 10 = 5x2, 5 subsequences of size 2 each)
nSeq = 1
nSteps = 3

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainCNN = XTrainList.reshape((XTrainList.shape[0], nSeq, nSteps, nFeatures))
XValidCNN = XValidList.reshape((XValidList.shape[0], nSeq, nSteps, nFeatures))
XTestCNN = XTestList.reshape((XTestList.shape[0], nSeq, nSteps, nFeatures))

# print(XTrainCNN.shape)
# print(XValidCNN.shape)
# print(XTestCNN.shape)

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainCNN, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValidCNN, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainCNN, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTestCNN, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 4.621769392519184
Updating best MAPE to 4.621769392519184...
Updating best seed to 0...
1 2.4118895713017063
Updating best MAPE to 2.4118895713017063...
Updating best seed to 1...
2 2.84393274811654
3 4.2295395444458945
4 4.29949095061294
5 1.6494250625471256
Updating best MAPE to 1.6494250625471256...
Updating best seed to 5...
6 4.129557633604657
7 1.2697080155621276
Updating best MAPE to 1.2697080155621276...
Updating best seed to 7...
8 0.9445348219108892
Updating best MAPE to 0.9445348219108892...
Updating best seed to 8...
9 7.0842747891923965
10 6.3849491566306895
11 3.7954876800145017
12 2.513543435844336
13 3.7055098790857026
14 6.377041773749667
15 4.965379283744148
16 3.5785587500618368
17 4.053540378438855
18 7.058278252793795
19 2.114203625779178
20 4.7451049005909685
21 6.108182251016054
22 3.3910071653595915
23 2.8596215514391723
24 3.57014384510745
25 1.4815041660307517
26 0.8405036871679727
Updating best MAPE to 0.8405036871679727...
Updating best seed to 26...
27 5.

# ConvLSTM

In [10]:
from tensorflow.keras.layers import ConvLSTM2D

# Number of subsequences to break X into (we do 10 = 5x2, 5 subsequences of size 2 each)
nSeq = 1
nSteps = 3
# Each input is rows x columns, we have rows=1 and columns=nSteps

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainConv = XTrainList.reshape((XTrainList.shape[0], nSeq, 1, nSteps, nFeatures))
XValidConv = XValidList.reshape((XValidList.shape[0], nSeq, 1, nSteps, nFeatures))
XTestConv = XTestList.reshape((XTestList.shape[0], nSeq, 1, nSteps, nFeatures))

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
    model.add(Flatten())
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainConv, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValidConv, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
model.add(Flatten())
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainConv, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTestConv, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 0.2558075512316829
Updating best MAPE to 0.2558075512316829...
Updating best seed to 0...
1 1.3930021000572799
2 1.0104786970137951
3 2.2548518586307154
4 1.6287350819092723
5 0.7649169816009171
6 1.7475062187232875
7 1.6054621191905154
8 0.4031907203497339
9 0.13965992935275778
Updating best MAPE to 0.13965992935275778...
Updating best seed to 9...
10 0.8541175033555625
11 2.327156401698172
12 1.2770199581660246
13 0.45023863000191644
14 0.5374787403430342
15 0.17125831432233102
16 0.20863672426419708
17 2.598113939244932
18 0.27191930498335987
19 0.22606682960483515
20 2.297944295244104
21 0.8260434094402583
22 0.7153509345871327
23 1.1269426374693197
24 1.4367980870230088
25 2.096008160846965
26 1.1893920351499327
27 2.8131302799053497
28 0.15343384456565423
29 0.7545682034523349
30 6.97427311542864
31 0.4462147618613433
32 0.590863665476309
33 0.25684091947533183
34 0.7507372146915593
35 0.28433793899495985
36 5.553014446720674
37 0.44171838202235375
38 1.3899125817438265
39 1.52